In [1]:
#imports
library(readr)
library(tigerstats)
library(ggplot2)
library(gridExtra)
library("DESeq2")
library("vsn")
library("pheatmap")
library("RColorBrewer")
library(repr)

Le chargement a nécessité le package : abd

Le chargement a nécessité le package : nlme

Le chargement a nécessité le package : lattice

Le chargement a nécessité le package : grid

Le chargement a nécessité le package : mosaic

Registered S3 method overwritten by 'mosaic':
  method                           from   
  fortify.SpatialPolygonsDataFrame ggplot2


The 'mosaic' package masks several functions from core packages in order to add 
additional features.  The original behavior of these functions should not be affected by this.


Attachement du package : 'mosaic'


Les objets suivants sont masqués depuis 'package:dplyr':

    count, do, tally


L'objet suivant est masqué depuis 'package:Matrix':

    mean


L'objet suivant est masqué depuis 'package:ggplot2':

    stat


Les objets suivants sont masqués depuis 'package:stats':

    binom.test, cor, cor.test, cov, fivenum, IQR, median, prop.test,
    quantile, sd, t.test, var


Les objets suivants sont masqués depuis 'package:base'

In [69]:
mc <- read_tsv("morphological-counts.tsv")
rna <- read_tsv("RNA-read-counts.tsv")
reactome <- read_tsv("c2.cp.reactome.v7.5.1.symbols.gmt")

Rows: 136 Columns: 65
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: "\t"
chr  (1): SMPLID
dbl (64): Mophological.cluster.0, Mophological.cluster.1, Mophological.clust...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 56200 Columns: 138
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: "\t"
chr   (2): Name, Description
dbl (136): GTEX-11EM3-0126, GTEX-11EQ9-0626, GTEX-11LCK-0526, GTEX-11NSD-012...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message:
"One or more parsing issues, see `problems()` for details"
Rows: 1614 Columns: 13
-- Column specification ----------------------------------------

In [19]:
for (i in 1:136){
    mc[i,2:65] <- mc[i,2:65]/sum(mc[i,2:65])
} 


In [70]:
rna[, 3:138]


GTEX-11EM3-0126,GTEX-11EQ9-0626,GTEX-11LCK-0526,GTEX-11NSD-0126,GTEX-11O72-2326,GTEX-11P7K-0226,GTEX-11P81-0126,GTEX-11P82-0226,GTEX-11TT1-1126,GTEX-11VI4-0226,...,GTEX-1QP9N-0126,GTEX-1QPFJ-0626,GTEX-1QW4Y-0226,GTEX-1R9JW-0226,GTEX-1R9PM-0226,GTEX-1RAZA-0826,GTEX-1RAZQ-0926,GTEX-1RAZR-1126,GTEX-1S5ZU-0726,GTEX-1S83E-0726
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,1,0,0,0,0,0,3,0,0,...,0,0,0,0,0,0,1,0,1,1
122,174,85,115,197,263,119,125,103,135,...,111,183,124,74,191,251,246,91,169,162
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
3,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,1,0,0,0,0,2,0,...,0,0,0,1,0,0,0,0,0,0
1,1,1,1,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,1,1
1,5,1,0,0,2,1,3,1,2,...,0,1,0,0,0,1,0,1,1,1
0,1,0,2,1,2,1,1,0,1,...,0,0,0,1,0,1,2,7,3,1
3,1,5,3,0,1,3,0,1,0,...,0,5,2,0,4,0,0,0,1,0


In [36]:
mc$Mophological.cluster.0

[1] 0.002532287 0.008844953 0.010972222 0.008571429 0.023707195 0.019307832
  [7] 0.022530783 0.010499852 0.002383222 0.019071837 0.009195402 0.033145509
 [13] 0.020357385 0.030636833 0.020417221 0.029427993 0.019320661 0.023328523
 [19] 0.004332360 0.002043318 0.025719338 0.027510969 0.003170175 0.031587697
 [25] 0.002457185 0.013418903 0.012544987 0.019117647 0.025895639 0.007063238
 [31] 0.009886307 0.008972845 0.008665316 0.038830716 0.003296982 0.010782200
 [37] 0.002443594 0.005525460 0.002512037 0.006081214 0.016965277 0.018255578
 [43] 0.006495240 0.014177541 0.006503605 0.009328358 0.010541447 0.014649682
 [49] 0.016098966 0.004301340 0.018166804 0.009981285 0.006215722 0.022178850
 [55] 0.010682130 0.008021097 0.006224256 0.021103187 0.007277089 0.037134407
 [61] 0.018573164 0.007135102 0.014903565 0.006914595 0.029525032 0.025402047
 [67] 0.013337199 0.011497476 0.060039761 0.005138911 0.003099486 0.018865248
 [73] 0.021023572 0.007193068 0.009662601 0.023702923 0.025810615 0.001418440
 [79] 0.006963250 0.026713948 0.010015814 0.015495172 0.013029745 0.021328958
 [85] 0.013363755 0.008215376 0.029746145 0.004740479 0.027981555 0.014629308
 [91] 0.015233415 0.016171101 0.021794699 0.012995452 0.003196852 0.022658463
 [97] 0.016056441 0.001239465 0.015072852 0.017170989 0.021859236 0.012690596
[103] 0.002144695 0.003830289 0.011191420 0.014723615 0.027086289 0.013660213
[109] 0.007348384 0.009084556 0.002660119 0.005069708 0.011660548 0.024953789
[115] 0.012969697 0.023477784 0.012583893 0.018707778 0.023409166 0.010939423
[121] 0.032392762 0.016027937 0.018278302 0.015409570 0.031391880 0.019610308
[127] 0.002908130 0.001800504 0.001178811 0.003500000 0.018783431 0.032078103
[133] 0.013034854 0.015096177 0.036099138 0.001627560

In [78]:
dds = DESeqDataSetFromMatrix(rna[, 3:138],colData=mc, design = ~ Mophological.cluster.0)
dds <- DESeq(dds)
#res <- results(dds)
#res.05 <- results(dds, alpha = 0.05)
#resSig <- subset(res, padj < 0.05)
#print(rownames(resSig))


converting counts to integer mode

  the design formula contains one or more numeric variables with integer values,
  specifying a model with increasing fold change for higher values.
  did you mean for this to be a factor? if so, first convert
  this variable to a factor using the factor() function

  the design formula contains one or more numeric variables that have mean or
  standard deviation larger than 5 (an arbitrary threshold to trigger this message).
  Including numeric variables with large mean can induce collinearity with the intercept.
  Users should center and scale numeric variables in the design to improve GLM convergence.



In [81]:
res <- results(dds)
res.05 <- results(dds, alpha = 0.05)
resSig <- subset(res, padj < 0.05)
print(rownames(resSig))

NULL


In [33]:
ncol(rna[,3:138])
nrow(mc)

[1] 136

[1] 136

In [71]:
means <- rowMeans(rna[,3:138])
med <- median(means)
cond <- means > med

In [72]:
rna$cond <- cond
rna

Name,Description,GTEX-11EM3-0126,GTEX-11EQ9-0626,GTEX-11LCK-0526,GTEX-11NSD-0126,GTEX-11O72-2326,GTEX-11P7K-0226,GTEX-11P81-0126,GTEX-11P82-0226,...,GTEX-1QPFJ-0626,GTEX-1QW4Y-0226,GTEX-1R9JW-0226,GTEX-1R9PM-0226,GTEX-1RAZA-0826,GTEX-1RAZQ-0926,GTEX-1RAZR-1126,GTEX-1S5ZU-0726,GTEX-1S83E-0726,cond
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
ENSG00000223972.5,DDX11L1,0,1,0,0,0,0,0,3,...,0,0,0,0,0,1,0,1,1,FALSE
ENSG00000227232.5,WASH7P,122,174,85,115,197,263,119,125,...,183,124,74,191,251,246,91,169,162,TRUE
ENSG00000278267.1,MIR6859-1,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,FALSE
ENSG00000243485.5,MIR1302-2HG,3,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,FALSE
ENSG00000237613.2,FAM138A,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,FALSE
ENSG00000268020.3,OR4G4P,1,1,1,1,0,0,0,0,...,0,0,1,0,0,1,0,1,1,FALSE
ENSG00000240361.1,OR4G11P,1,5,1,0,0,2,1,3,...,1,0,0,0,1,0,1,1,1,FALSE
ENSG00000186092.4,OR4F5,0,1,0,2,1,2,1,1,...,0,0,1,0,1,2,7,3,1,FALSE
ENSG00000238009.6,RP11-34P13.7,3,1,5,3,0,1,3,0,...,5,2,0,4,0,0,0,1,0,TRUE


In [75]:
rna <- rna[rna$cond == TRUE,]

In [76]:
rna

Name,Description,GTEX-11EM3-0126,GTEX-11EQ9-0626,GTEX-11LCK-0526,GTEX-11NSD-0126,GTEX-11O72-2326,GTEX-11P7K-0226,GTEX-11P81-0126,GTEX-11P82-0226,...,GTEX-1QPFJ-0626,GTEX-1QW4Y-0226,GTEX-1R9JW-0226,GTEX-1R9PM-0226,GTEX-1RAZA-0826,GTEX-1RAZQ-0926,GTEX-1RAZR-1126,GTEX-1S5ZU-0726,GTEX-1S83E-0726,cond
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
ENSG00000227232.5,WASH7P,122,174,85,115,197,263,119,125,...,183,124,74,191,251,246,91,169,162,TRUE
ENSG00000238009.6,RP11-34P13.7,3,1,5,3,0,1,3,0,...,5,2,0,4,0,0,0,1,0,TRUE
ENSG00000233750.3,CICP27,4,2,11,11,2,11,6,1,...,2,12,13,0,2,8,7,6,0,TRUE
ENSG00000268903.1,RP11-34P13.15,174,73,225,163,313,87,72,96,...,179,154,130,47,41,227,79,119,74,TRUE
ENSG00000269981.1,RP11-34P13.16,114,69,145,146,134,70,57,50,...,134,120,132,25,27,212,37,122,49,TRUE
ENSG00000239906.1,RP11-34P13.14,0,1,10,0,13,1,9,0,...,11,4,3,2,0,11,1,0,0,TRUE
ENSG00000241860.6,RP11-34P13.13,35,18,63,22,63,23,77,9,...,27,7,2,41,14,35,18,66,79,TRUE
ENSG00000279928.2,ABC7-43046700E7.1,1,10,0,0,2,2,2,2,...,0,0,6,2,0,12,1,0,0,TRUE
ENSG00000279457.4,RP11-34P13.18,241,388,84,370,368,156,303,141,...,291,422,336,288,366,267,235,240,454,TRUE
